In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import pickle 
from datetime import datetime
import math 
from functions.predictor import process_parkrun_data_for_models, user_input, target_time

In [2]:
model_file_path = 'models/to_use/xgb_opt_model.pkl'
scaler_file_path = 'models/to_use/minmax_scaler.pkl'
data_for_model = 'data/clean/cleaned_parkrun_no_outliers.csv'

In [3]:
df = process_parkrun_data_for_models(data_for_model)

# Automatic input (with parkrun id or results link)


Do you know your parkrun id? 
Should be an integer following A. For example A-5125087

Your link for full results should be in the form:
https://www.parkrun.org.uk/parkrunner/5125087/all

# Manual input:

In [6]:
target_time(user_input(df), df, model_file_path, scaler_file_path)

Enter expected temperature in (°C). (Leave blank for a default value of 11.8):  5
Enter expected windspeed in km/h (Leave blank for a default value of 20.6):  12
Is it likely to rain? (y/n):  n
How many parkruns have you run previously? (Leave blank for a default value of 33.0):  28
Enter the planned parkrun date (leave blank for today, format YYYY-MM-DD):  
Enter the date of your last parkrun (format YYYY-MM-DD):  05-10-2024


Invalid date format. Please enter the date in the format YYYY-MM-DD.


Enter the date of your last parkrun (format YYYY-MM-DD):  2024-10-05
Enter the rough date you started doing parkruns (leave blank for 2022-05-13, format YYYY-MM-DD):  2018-6-6
Enter your most recent parkrun time in the form 'mins, secs' (Required):  27,44
Enter your previous PB in the form 'mins, secs'. Leave blank to use your previous time 27.7 mins:  23,51
Enter your average parkrun time in the form 'mins, secs'. Leave blank to use your previous time 27.7 mins:  28,32
Enter your age:  30
Enter your gender (m/f) or leave blank:  m



Data added successfully
Target time: 28m2s


28.032959238688154